# LCEL 의 기본 구조
1. 프롬프트 템플릿
2. 모델
3. 출력 형태

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "langchain_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

# 1. 프롬프트 템플릿 설정하기

In [16]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{today_year} 기준으로 최근 {during_year}년에 {country}에 오는 관광객 수는?"
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country', 'during_year', 'today_year'], input_types={}, partial_variables={}, template='{today_year} 기준으로 최근 {during_year}년에 {country}에 오는 관광객 수는?')

In [18]:
# prompt 생성
prompt = prompt_template.format(during_year = "5", today_year = '2025', country = '한국')
prompt

'2025 기준으로 최근 5년에 한국에 오는 관광객 수는?'

# 2. 모델 선택하기

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

# 프롬프트와 모델 연결하기

In [9]:
chain = prompt_template | model
chain

PromptTemplate(input_variables=['year'], input_types={}, partial_variables={}, template='최근 {year}년에 한국에 오는 관광객 수는?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000028278B596C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002827B2237F0>, root_client=<openai.OpenAI object at 0x000002827ACCFF40>, root_async_client=<openai.AsyncOpenAI object at 0x000002827B223760>, model_name='gpt-4.1-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [10]:
input = {'year' : 5}
answer = chain.invoke(input)
print(answer.content)

최근 5년간 한국을 방문한 관광객 수에 대해 알려드리겠습니다. 다만, 2020년부터 시작된 코로나19 팬데믹으로 인해 관광객 수가 크게 변동했음을 참고해 주세요.

- 2018년: 약 1,540만 명
- 2019년: 약 1,760만 명 (역대 최고치)
- 2020년: 약 250만 명 (코로나19 영향으로 급감)
- 2021년: 약 30만 명 (여전히 제한적)
- 2022년: 약 200만 명 (점진적 회복 시작)

2023년과 2024년 데이터는 아직 공식 통계가 완전히 집계되지 않았으나, 방역 완화와 국제 여행 재개로 인해 관광객 수가 점차 회복 중인 것으로 알려져 있습니다.

보다 정확한 최신 통계는 한국관광공사나 통계청 공식 자료를 참고하시길 권장드립니다.


# 3. 출력 양식 정해보기

In [11]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser, CommaSeparatedListOutputParser

outputparser = StrOutputParser()

# 모두 연결하기

In [21]:
chain = prompt_template | model | outputparser

input = {'during_year' : "3", 'today_year' : '2025', 'country' : '한국'}
answer = chain.invoke(input)
print(answer)

2025년 기준으로 최근 3년간(2022년, 2023년, 2024년) 한국을 방문한 관광객 수에 대한 최신 공식 통계는 아직 완전히 집계되지 않았을 수 있습니다. 다만, 한국관광공사와 관련 정부 기관에서 발표한 자료를 참고하면 다음과 같은 추세를 알 수 있습니다.

- **2022년**: 코로나19 팬데믹 이후 국제 관광이 점차 회복되기 시작한 해로, 약 1,000만 명 내외의 외국인 관광객이 한국을 방문한 것으로 추정됩니다.
- **2023년**: 방역 조치 완화와 국제 여행 수요 증가로 관광객 수가 크게 증가하여 약 1,800만 명에서 2,000만 명 사이로 예상됩니다.
- **2024년**: 관광 회복세가 지속되어 2,500만 명 이상이 한국을 방문했을 가능성이 높습니다.

정확한 수치는 한국관광공사 공식 홈페이지나 통계청 자료를 통해 확인하는 것이 가장 정확합니다. 필요하시면 최신 공식 통계 발표가 나오는 시점에 다시 확인해 드릴 수 있습니다.


---

# 실습

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# template 정의
template = """
너는 지금 {animal}이고, 직업은 {role}야.

1. 네가 하는 일을 한 문장으로 설명해줘.
2. 청중이 웃으면서 공감할 수 있도록 유머러스하게 말해줘.
"""
prompt = PromptTemplate.from_template(template)
prompt

# 모델 불러오기
model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

# 출력 파서 정하기
outputparser = StrOutputParser()

chain = prompt | model | outputparser

answer = chain.invoke({'animal' : '고양이', 'role' : '개발자'})

In [24]:
print(answer)

1. 나는 코드를 쥐 잡듯이 잡아서 버그라는 쥐를 사냥하는 고양이 개발자야.

2. 매일 키보드 위를 폴짝폴짝 뛰어다니면서, 버그라는 쥐를 잡으려고 노력하는데… 가끔은 내가 쥐가 되어버려서 디버깅이라는 미로에서 헤매기도 하지! 냥~ 코드도 쥐처럼 빠르고 날렵해야 한다고 믿는 고양이 개발자라구!


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# template 정의
template = """
너는 10년차 영어 회화 선생님이야
학생들을 위한 영어 대화 지문을 만들거야
주어진 상황과 난이도에 맞게 영어 지문을 만들어줘

#난이도
{level}

#상황
{situation}

#FORMAT
A : 대사,
B : 대사,
...
반복
"""
prompt = PromptTemplate.from_template(template)
prompt

# 모델 불러오기
model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

# 출력 파서 정하기
outputparser = StrOutputParser()

chain = prompt | model | outputparser

answer = chain.invoke({'situation' : '학교에서 kim 과 mike 가 자기소개를 하는 상황', 'level' : '하'})
print(answer)